In [1]:
import json, os, joblib, sys
import pandas as pd, numpy as np

sys.path.insert(0, os.path.abspath('../'))
from assist.utils import normalScaler

In [2]:
dataframe = pd.read_csv(f"{os.path.abspath('../')}/data/raw/MCP_data.csv")
dataframe = dataframe[['Game Name', 'Actual Bidding Date', 'Proximity', 'MCP']]
dataframe.columns = ['simulation', 'timestamp', 'proximity', 'mcp']
dataframe['timestamp'] = dataframe['timestamp'].astype('datetime64[h]')

In [3]:
dst = '/home/suraj/Desktop/nbeats/data/mcp'
os.makedirs(f'{dst}/train', exist_ok=True)
os.makedirs(f'{dst}/test', exist_ok=True)

dataframe['timestamp'] = dataframe['timestamp'].astype('datetime64[h]')
mcpbook = {}
for i in range(24):
    mcpbook[f'mcp_{i+1}'] = []
    mcpbook[f'mcp_{i+1}_EWA'] = []
    mcpbook[f'mcp_{i+1}_GWA'] = []

for sim in dataframe['simulation'].unique():
    miniframe = dataframe[dataframe['simulation']==sim].reset_index(drop=True)
    miniframe = miniframe.pivot(index='timestamp', columns='proximity', values='mcp').reset_index()
    miniframe.columns = ['timestamp'] + [f'mcp_{i+1}' for i in range(24)]
    
    for i in range(24):
        miniframe[f'mcp_{i+1}_EWA'] = miniframe[f'mcp_{i+1}'].ewm(alpha = 0.6).mean()
        miniframe[f'mcp_{i+1}_GWA'] = miniframe[f'mcp_{i+1}'].rolling(window=5, win_type='gaussian', min_periods=1, center=True).mean(std=1)
    
    miniframe.set_index('timestamp', inplace=True)
    miniframe = miniframe.resample('H').interpolate()
    miniframe = miniframe.reset_index()
    
    nums = int(0.8*(len(miniframe)))
    trnframe = miniframe.head(nums)
    tstframe = miniframe.tail(len(miniframe)-nums+168)
    
    for i in range(24):
        mcpbook[f'mcp_{i+1}'] = mcpbook[f'mcp_{i+1}'] + trnframe[f'mcp_{i+1}'].tolist()
        mcpbook[f'mcp_{i+1}_EWA'] = mcpbook[f'mcp_{i+1}_EWA'] + trnframe[f'mcp_{i+1}_EWA'].tolist()
        mcpbook[f'mcp_{i+1}_GWA'] = mcpbook[f'mcp_{i+1}_GWA'] + trnframe[f'mcp_{i+1}_GWA'].tolist()
    
    trnframe.to_csv(f'{dst}/train/{sim}.csv', index=False)
    tstframe.to_csv(f'{dst}/test/{sim}.csv', index=False)
    
scalebook = {}
for name, data in mcpbook.items():
    data = np.array(data)
    scalebook[name] = normalScaler(mean=data.mean(), std=data.std())
joblib.dump(scalebook, f'{dst}/scalebook')

['/home/suraj/Desktop/nbeats/data/mcp/scalebook']